<a href="https://colab.research.google.com/github/sonyharianto1991/multicriteria_healthcheck/blob/main/baseline_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 🚀 AUTOMATED HEALTH CHECK & ROLLBACK SYSTEM (Prototype)
# Baseline Model + Multi-Criteria Evaluation
# ==========================================

!pip install torch torchvision albumentations timm scikit-learn tqdm numpy scipy pandas matplotlib -q

import torch, torchvision, numpy as np, pandas as pd, matplotlib.pyplot as plt
from torchvision import transforms, datasets, models
from torch import nn, optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm
from scipy.stats import ks_2samp, entropy
from PIL import Image, ImageEnhance, ImageFilter

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

# ==========================================
# 1️⃣ DATASET SETUP
# ==========================================
data_dir = "/content/drive/MyDrive/TESIS/dataset/"  # ubah sesuai folder di Google Drive
train_dir = f"{data_dir}/train" ## 611 files
val_dir   = f"{data_dir}/val" ## 76 files
test_dir  = f"{data_dir}/test" ## 76 files

# Transformasi
train_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_ds = datasets.ImageFolder(train_dir, transform=train_tfms)
val_ds   = datasets.ImageFolder(val_dir, transform=val_tfms)
test_ds  = datasets.ImageFolder(test_dir, transform=val_tfms)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_dl   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)
test_dl  = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2)

print("📂 Classes:", train_ds.classes)

# ==========================================
# 2️⃣ BASELINE TRAINING (MobileNetV3)
# ==========================================
model = models.mobilenet_v3_small(weights='IMAGENET1K_V1')
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(train_ds.classes))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(5):
    model.train()
    running_loss = 0
    for x, y in tqdm(train_dl, desc=f"Epoch {epoch+1}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward(); optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_dl):.4f}")

torch.save(model.state_dict(), "mobilenet_baseline.pth")
print("💾 Model saved as mobilenet_baseline.pth")

# ==========================================
# 3️⃣ BASELINE EVALUATION
# ==========================================
model.eval(); y_true, y_pred, y_prob = [], [], []
with torch.no_grad():
    for x, y in test_dl:
        x = x.to(device)
        out = torch.softmax(model(x), dim=1)
        pred = out.argmax(1).cpu().numpy()
        y_true.extend(y.numpy()); y_pred.extend(pred)
        y_prob.extend(out.cpu().numpy())

acc = accuracy_score(y_true, y_pred)
f1  = f1_score(y_true, y_pred, average='macro')
print(f"📊 Baseline Accuracy: {acc:.4f}, F1: {f1:.4f}")

confidences = [p.max() for p in y_prob]
np.save("baseline_conf.npy", confidences)
pd.DataFrame({'y_true':y_true,'y_pred':y_pred,'conf':confidences}).to_csv("baseline_eval.csv",index=False)
print("📁 Saved baseline_conf.npy and baseline_eval.csv")

# ==========================================
# 4️⃣ MULTI-CRITERIA METRICS
# ==========================================
def population_stability_index(expected, actual, bins=10):
    def scale_range(input, min=0, max=1):
        input += -(np.min(input))
        input /= np.max(input) / (max - min)
        input += min
        return input
    expected, actual = scale_range(np.array(expected)), scale_range(np.array(actual))
    breakpoints = np.arange(0, bins+1) / bins
    expected_perc, _ = np.histogram(expected, breakpoints)
    actual_perc, _  = np.histogram(actual, breakpoints)
    expected_perc = expected_perc / len(expected)
    actual_perc = actual_perc / len(actual)
    psi = np.sum((expected_perc - actual_perc) * np.log((expected_perc + 1e-8) / (actual_perc + 1e-8)))
    return psi

def kl_divergence(p, q, bins=50):
    p_hist, _ = np.histogram(p, bins=bins, density=True)
    q_hist, _ = np.histogram(q, bins=bins, density=True)
    p_hist += 1e-10; q_hist += 1e-10
    p_hist /= np.sum(p_hist); q_hist /= np.sum(q_hist)
    return entropy(p_hist, q_hist)

def aggregate_multi_metrics(y_true, y_pred, conf_base, conf_now, w=[0.2,0.15,0.15,0.15,0.2,0.15]):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    psi = population_stability_index(conf_base, conf_now)
    kld = kl_divergence(conf_base, conf_now)
    acc_r = 1 - acc; prec_r = 1 - prec; rec_r = 1 - rec; f1_r = 1 - f1
    psi_r = min(psi/0.25,1); kld_r = min(kld/1.0,1)
    score = np.dot(w, [acc_r,prec_r,rec_r,f1_r,psi_r,kld_r])
    return {"acc":acc,"prec":prec,"rec":rec,"f1":f1,"psi":psi,"kld":kld,"score":score}

# ==========================================
# 5️⃣ DRIFT SIMULATION & HEALTH CHECK
# ==========================================
# Reuse baseline model
model = models.mobilenet_v3_small()
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, 2)
model.load_state_dict(torch.load("mobilenet_baseline.pth", map_location=device))
model = model.to(device).eval()

test_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
test_ds = datasets.ImageFolder(test_dir, transform=test_tfms)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

# Simulate drift: brightness + blur
def apply_drift(img):
    img = img.convert("RGB")
    img = ImageEnhance.Brightness(img).enhance(0.6)
    img = img.filter(ImageFilter.GaussianBlur(radius=2))
    return img

# Apply drift simulation (optional)
# You can rebuild dataset with drifted images if needed

base_conf = np.load("baseline_conf.npy")

y_true, y_pred, conf_now = [], [], []
with torch.no_grad():
    for (x,y) in tqdm(test_dl, desc="Evaluating Drifted Model"):
        x = x.to(device)
        probs = torch.softmax(model(x), dim=1)
        y_pred.extend(probs.argmax(1).cpu().numpy())
        y_true.extend(y.numpy())
        conf_now.extend(probs.max(1)[0].cpu().numpy())

results = aggregate_multi_metrics(y_true, y_pred, base_conf, conf_now)
print("\n📈 Multi-Criteria Evaluation Results:")
print(pd.Series(results))

status = "⚠️ DEGRADE" if results['score'] >= 0.7 else "✅ OK"
print(f"\nFinal Model Health Status: {status} | Score = {results['score']:.3f}")

✅ Device: cpu
📂 Classes: ['1', '2']
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 61.9MB/s]
Epoch 1: 100%|██████████| 17/17 [00:53<00:00,  3.15s/it]


Epoch 1 Loss: 0.6166


Epoch 2: 100%|██████████| 17/17 [00:35<00:00,  2.10s/it]


Epoch 2 Loss: 0.5263


...

Epoch 5: 100%|██████████| 17/17 [00:35<00:00,  2.06s/it]

Epoch 5 Loss: 0.3560
💾 Model saved as mobilenet_baseline.pth


📊 Baseline Accuracy: 0.7895, F1: 0.6833
📁 Saved baseline_conf.npy and baseline_eval.csv


Evaluating Drifted Model: 100%|██████████| 3/3 [00:02<00:00,  1.22it/s]


📈 Multi-Criteria Evaluation Results:
acc      0.789474
prec     0.832504
rec      0.664479
f1       0.683333
psi      0.000000
kld      0.000000
score    0.165058
dtype: float64

Final Model Health Status: ✅ OK | Score = 0.165
